In [1]:
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from data.base_data_module import BaseDataset

C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [2]:
def scale_keypoints(keypoints, max_size=(256, 256)):
    # divide all x and y coordinates by the corresponding axis max
    # this is defined on the image processing stage
    keypoints[:, :, 0] = keypoints[:, :, 0]/max_size[0]
    keypoints[:, :, 1] = keypoints[:, :, 1]/max_size[1]
    
    return keypoints

In [3]:
# train_dataset = np.load("dataset/Train.npy", allow_pickle=True)
# test_dataset = np.load("dataset/Test.npy", allow_pickle=True)
# val_dataset = np.load("dataset/Val.npy", allow_pickle=True)

train_keypoints = np.load("dataset/Train_keypoints.npy",  allow_pickle=True)
test_keypoints = np.load("dataset/Test_keypoints.npy",  allow_pickle=True)
val_keypoints = np.load("dataset/Val_keypoints.npy",  allow_pickle=True)

train_mfccs = torch.load("dataset/Train_mfccs.pt" )
test_mfccs = torch.load("dataset/Test_mfccs.pt" )
val_mfccs = torch.load("dataset/Val_mfccs.pt" )

train_set  = BaseDataset(data=train_mfccs, targets=train_keypoints, target_transform=scale_keypoints)


In [4]:
train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)
val_loader = DataLoader(val_dataset)

NameError: name 'train_dataset' is not defined

In [8]:
# Here we define our model as a class
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

model = LSTM(lstm_input_size, h1, batch_size=num_train, output_dim=output_dim, num_layers=num_layers)

NameError: name 'nn' is not defined

In [9]:
loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

#####################
# Train model
#####################

hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

C:\Users\Filipe\.conda\envs\wav2kp\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


NameError: name 'model' is not defined

In [7]:
# TODO: make these args from the argparser
trainer = pl.Trainer(gpus=1)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [6]:
 mfcc, kp = train_set[1]

In [7]:
mfcc.shape

torch.Size([5300, 40])